In [1]:
from encodingUtils import encode, PlaceCellEncoder2D
theta0=90

sigma_res_noise = 0.005
sigma_S = 0.002

spatial_res = 5

gamma_pg=0.5

gradient_spectral_rad = .5

description='rho=' + str(gradient_spectral_rad) + '_5'

grid_size = spatial_res  
sigma = 0.2 

enocder = PlaceCellEncoder2D(grid_size, sigma)

In [2]:
from envAgent import Environment, LinearAgent
import numpy as np

# BUG: Agent in the environment was not designed to be called
# as any other variable BUT agent, this caused problems (maybe) with agent_res
env = Environment()
agent = LinearAgent()

agent.weights_0 = np.copy(agent.weights)

In [3]:
from goal_onr import RESERVOIRE_SIMPLE_NL_MULT

# Changed time from 600 to 1200
N, I, O, TIME = 2000, 15, 7, 30
shape = (N, I, O, TIME)

dt = 1.0/ TIME 
tau_m_f = 20. * dt
tau_m_s = 20. * dt
tau_s = 2. * dt
tau_ro = .001 * dt
beta_s  = np.exp (-dt / tau_s)
beta_ro = np.exp (-dt / tau_ro)
sigma_teach = 0.
sigma_input = .06
sigma_rec = 0.5/np.sqrt(N)
offT = 1
dv = 5.
alpha_rout = .0005 
alpha_pg = 0.0005
alpha = 0.
Vo = 0
h = 0
s_inh = 0
sigma_output = .1

n_electrodes = 85
n_pca = n_electrodes

par = {'tau_m_f' : tau_m_f,'tau_m_s' : tau_m_s, 'tau_s' : tau_s, 'tau_ro' : tau_ro, 'beta_ro' : beta_ro,
    'dv' : dv, 'alpha' : alpha, 'Vo' : Vo, 'h' : h, 's_inh' : s_inh,
    'N' : N, 'T' : TIME, 'dt' : dt, 'offT' : offT, 'alpha_rout' : alpha_rout,
    'sigma_input' : sigma_input, 'sigma_teach' : sigma_teach,'sigma_rec' : sigma_rec, 'shape' : shape,'sigma_output':sigma_output}

N = 500
# Change to 1 if you want to use the 1 - strategy
gamma_grad = 1.5
input_dim = spatial_res**2 + 4 + 10
shape = ( N , input_dim , 4*spatial_res**2 , TIME )
par['shape']=shape
par['sigma_input'] = sigma_input
par['sigma_rec'] = gradient_spectral_rad/np.sqrt(N)
par['tau_m_s'] = .00*dt
par['tau_m_f'] = .00*dt
network_reservoire_gradient = RESERVOIRE_SIMPLE_NL_MULT (par)
network_reservoire_gradient.Jin_mult = np.random.normal(0,0.1,size=(N,))

In [4]:
ERRORS = []

alpha_w = 0.05

r_traj = []
e_traj = []
dw_traj = []
r_targ = []
y_data = []
S = []

x_net_coll = []
y_coll = []

x_grad_net_coll = []
y_grad_coll = []

x_w_coll = []
y_w_coll = []

In [ ]:
change_every = 100
n_resets = 2

eta = 0.008 * 0.125

dw_filt = 0
n_angle = 0

total_rewards = []
total_reservoire_rates = []
total_gradients = []

for n in range(n_resets):
    theta0 = 45 * n_angle
    n_angle += 1 

    env.reset(.5,.5, theta0=theta0)
    agent.weights = np.copy(agent.weights_0 * 0)

    rewards = []
    reservoire_rates = []
    gradients = []

    for episode in range(change_every):
        done = False 
        time = 0

        env.reset_inner()
        network_reservoire_gradient.reset()
        agent.update_filt = np.copy(agent.weights_0 * 0)

        reservoire_rate = []
        gradient = []
        
        while not done:
            time += 1
            if time >= TIME:
                break
            
            state = np.copy(env.agent_position)
            state_enc = enocder.encode_position(state)
            logits = agent.weights @ state_enc
            action_probs = np.exp(logits) / np.sum(np.exp(logits))
            action = np.random.choice(len(action_probs), p=action_probs)

            reward, done = env.step(action)
            weights_old = np.copy(agent.weights)
            probs = np.copy(action_probs)
            probs[action] -= 1
            agent.updates_filt = agent.update_filt*gamma_pg + (1.-gamma_pg)* np.outer(probs,state_enc)
            dw_out = np.copy(-reward * agent.learning_rate * agent.update_filt) * eta
            agent.weights += dw_out

            r_encoded = encode(reward)

            input_modulation = .1 + gamma_grad * network_reservoire_gradient.Jin_mult * reward
            input_modulation = input_modulation.flatten()

            reservoire_input = np.concatenate((state_enc, probs, r_encoded.flatten()))
            network_reservoire_gradient.step_rate(reservoire_input, input_modulation, 0)

            reservoire_rate.append(network_reservoire_gradient.S)
            gradient.append(dw_out.flatten())

        reservoire_rates.append(np.array(reservoire_rate))
        gradients.append(np.array(gradient))
        rewards.append(reward)  

    total_reservoire_rates.append(reservoire_rates)
    total_gradients.append(gradients)
    total_rewards.append(rewards)      
        
print(np.array(total_rewards).shape)
print(total_reservoire_rates.shape)

c:\Users\marcu\OneDrive\Desktop\Stuff\github_repos\AML2025\FinalProject\Testing\goal_onr.py:131: RuntimeWarning: divide by zero encountered in divide
  self.H   = self.H   * np.exp(-self.dt/self.tau_m) + (1-np.exp(-self.dt/self.tau_m) ) * np.tanh( ( self.J @ self.S_hat   + self.Jin @ inp )* (inp_modulation) )


(2, 100)


AttributeError: 'list' object has no attribute 'shape'